Resources about numpy:
https://numpy.org/doc/stable/user/quickstart.html

Resources about pandas:
https://pandas.pydata.org/docs/reference/frame.html


In [233]:
!pip install numpy

In [234]:
!pip3 install pandas

In [235]:
!pip install openpyxl

In [236]:
# Import numpy and pandas
import numpy as np
import pandas as pd

In [237]:
# TODO: Create function of code below called system_reader which takes the file path of the system data as input
# and returns the line and bus data.

In [238]:
# Replace 'file.xlsx' with your actual file path
file_path = 'system_basecase.xlsx'

# Replace 'Sheet1' with the name of the sheet you want to read
sheet_name1 = 'BusData'
sheet_name2 = 'LineData'


# Read the Excel file into a pandas DataFrame for the specified sheet
bus_data = pd.read_excel(file_path, sheet_name=sheet_name1)
line_data = pd.read_excel(file_path, sheet_name=sheet_name2)

# Display the contents of the DataFrame
print(bus_data)
print(line_data)

    Bus #  P MW  Q MVAr Type  P Gen  V Set
0       1   0.0     0.0    S      0  1.040
1       2  20.1    12.5    D      0  1.000
2       3  45.2    16.0    G     42  1.025
3       4  60.8    10.9    D      0  1.000
4       5   7.6    -1.6    D      0  1.000
5       6  13.5     8.5    D      0  1.000
6       7  33.5    11.6    D      0  1.000
7       8   9.0     5.8    D      0  1.000
8       9   4.3     2.1    G     23  1.020
9      10   5.2     1.6    G     27  1.020
10     11  13.5     5.8    D      0  1.000
11     12  14.9     5.0    G     33  1.030
    From  To  Rtotal, p.u.  Xtotal, p.u.  Btotal, p.u.  Fmax, MVA
0      1   2       0.01938       0.05917        0.0528         95
1      1   5       0.05203       0.20304        0.0492        100
2      2   3       0.04699       0.19797        0.0438       9999
3      2   4       0.05811       0.17632        0.0340       9999
4      2   5       0.05695       0.07388        0.0346       9999
5      3   4       0.67010       0.17103     

In [239]:
# TODO: Create function of the code below called admittance_matrix

In [240]:
# Calculating admittance matrix
num_buses = len(bus_data)
Ybus = np.zeros((num_buses, num_buses), dtype=complex)

# Test admitance matrix
# print(Ybus)

# Calculate total admittance of lines connected to each bus and store on the main diagonal
for i in range(num_buses):
    # Debug code
    # print('iteration', i)
    
    # Select all lines connected to the current bus in question.
    bus_lines = line_data[(line_data['From'] == i + 1) | (line_data['To'] == i + 1)]
    print(bus_lines)
    # Sum all line admittances.
    total_admittance = sum(1 / (line['Rtotal, p.u.'] + 1j * line['Xtotal, p.u.']) for _, line in bus_lines.iterrows())
    Ybus[i, i] = total_admittance

# Test calculations
# print(Ybus)

# Calculate admittance between each pair of buses and fill the off-diagonal elements
for _, line in line_data.iterrows():
    from_bus = int(line['From']) - 1
    to_bus = int(line['To']) - 1
    
    admittance = 1 / (line['Rtotal, p.u.'] + 1j * line['Xtotal, p.u.'])
    
    Ybus[from_bus, to_bus] = -admittance
    Ybus[to_bus, from_bus] = -admittance  # Assuming symmetric matrix

# print("Admittance Matrix (Ybus):")
print(Ybus)

   From  To  Rtotal, p.u.  Xtotal, p.u.  Btotal, p.u.  Fmax, MVA
0     1   2       0.01938       0.05917        0.0528         95
1     1   5       0.05203       0.20304        0.0492        100
   From  To  Rtotal, p.u.  Xtotal, p.u.  Btotal, p.u.  Fmax, MVA
0     1   2       0.01938       0.05917        0.0528         95
2     2   3       0.04699       0.19797        0.0438       9999
3     2   4       0.05811       0.17632        0.0340       9999
4     2   5       0.05695       0.07388        0.0346       9999
   From  To  Rtotal, p.u.  Xtotal, p.u.  Btotal, p.u.  Fmax, MVA
2     2   3       0.04699       0.19797        0.0438       9999
5     3   4       0.67010       0.17103        0.0128       9999
   From  To  Rtotal, p.u.  Xtotal, p.u.  Btotal, p.u.  Fmax, MVA
3     2   4       0.05811       0.17632        0.0340       9999
5     3   4       0.67010       0.17103        0.0128       9999
6     4   5       0.01335       0.04211        0.0391       9999
7     4   7       0.00000

In [241]:
# # Create 12x1 Matrix to store given V values
# num_buses = len(bus_data)
# V_bus = np.zeros(num_buses)
# # print(num_buses)
# for i in range(len(bus_data)):
#     V_bus[i] = bus_data.iloc[i]['V Set']
# # print(V_bus)

In [242]:
V_bus = np.ones(num_buses)  # Default 1.0 p.u. for all buses
theta = np.zeros(num_buses)  # Default 0 for all buses

# Set specific values for Slack and PV buses
for i in range(num_buses):
    bus_type = bus_data.iloc[i]['Type']
    if bus_type == 'S':  # Slack bus
        V_bus[i] = bus_data.iloc[i]['V Set']  # Set specified voltage magnitude
        theta[i] = 0  # Slack bus angle is typically considered as the reference (0 degrees or radians)
    elif bus_type == 'G':
        V_bus[i] = bus_data.iloc[i]['V Set']  # Set specified voltage magnitude
        # Angle remains 0 (or set to the slack bus angle if known)

In [243]:
base_MVA = 100  # Base MVA

# Create a matrix to store per-unit values for active power (P), reactive power (Q), and generated power (P Gen)
per_unit_matrix = np.zeros((num_buses, 3))  # 3 columns for P, Q, and P Gen

# Calculate and store per-unit values in the matrix
for i in range(num_buses):
    P_pu = bus_data.iloc[i]['P MW'] / base_MVA
    Q_pu = bus_data.iloc[i]['Q MVAr'] / base_MVA
    P_Gen_pu = bus_data.iloc[i]['P Gen'] / base_MVA
    
    # Store per-unit values in the matrix
    per_unit_matrix[i, 0] = P_pu
    per_unit_matrix[i, 1] = Q_pu
    per_unit_matrix[i, 2] = P_Gen_pu

# Display the matrix
print("Per-Unit Matrix (Active Power (P), Reactive Power (Q), Generated Power (P Gen)):")
print(per_unit_matrix)

Per-Unit Matrix (Active Power (P), Reactive Power (Q), Generated Power (P Gen)):
[[ 0.     0.     0.   ]
 [ 0.201  0.125  0.   ]
 [ 0.452  0.16   0.42 ]
 [ 0.608  0.109  0.   ]
 [ 0.076 -0.016  0.   ]
 [ 0.135  0.085  0.   ]
 [ 0.335  0.116  0.   ]
 [ 0.09   0.058  0.   ]
 [ 0.043  0.021  0.23 ]
 [ 0.052  0.016  0.27 ]
 [ 0.135  0.058  0.   ]
 [ 0.149  0.05   0.33 ]]


In [244]:
#Calculating the net injection

df = pd.DataFrame(per_unit_matrix, columns=['P, p.u.', 'Q, p.u.', 'P_Gen, p.u.'])

# Perform column operations
netInjection_matrix = df.copy()
netInjection_matrix['P, p.u.'] = netInjection_matrix['P_Gen, p.u.'] - netInjection_matrix['P, p.u.']
netInjection_matrix['Q, p.u.'] = -1 * netInjection_matrix['Q, p.u.']

# Rename columns in the new matrix
netInjection_matrix.columns = ['P_net, p.u.', 'Q_net, p.u.', 'P_Gen, p.u.']

print(netInjection_matrix)


    P_net, p.u.  Q_net, p.u.  P_Gen, p.u.
0         0.000       -0.000         0.00
1        -0.201       -0.125         0.00
2        -0.032       -0.160         0.42
3        -0.608       -0.109         0.00
4        -0.076        0.016         0.00
5        -0.135       -0.085         0.00
6        -0.335       -0.116         0.00
7        -0.090       -0.058         0.00
8         0.187       -0.021         0.23
9         0.218       -0.016         0.27
10       -0.135       -0.058         0.00
11        0.181       -0.050         0.33


In [245]:
# Assuming Ybus is the calculated admittance matrix and bus_data contains bus information

# x @ y for matric multiplication

# Initialize arrays to store calculated P and Q
P_calc = np.zeros(num_buses)
Q_calc = np.zeros(num_buses)
#Has to be theta I - theta j

# Calculate P and Q injections at each bus with zero angles assumption
   
for i in range(num_buses):
    # Compute P and Q injections based on the admittance matrix and assuming zero angles
    P_calc[i] = V_bus[i] * (np.sum(V_bus[j] * ((np.real(Ybus[i, j])) * np.cos(theta[i]-theta[j]) \
                                       + np.imag(Ybus[i,j]) * np.sin(theta[i]-theta[j]))for j in range(num_buses)))
    
    Q_calc[i] = V_bus[i] * (np.sum(V_bus[j] * ((np.real(Ybus[i, j])) * np.sin(theta[i]-theta[j]) \
                                       - np.imag(Ybus[i,j]) * np.cos(theta[i]-theta[j]))for j in range(num_buses)))
  


# Calculate mismatches  dP = P_k - P_net
mismatch_P = P_calc - netInjection_matrix['P_net, p.u.'] 

# Calculate mismatches  dQ = Q_k - Q_net
mismatch_Q = Q_calc - netInjection_matrix['Q_net, p.u.']



# Display mismatches
print("Mismatch in Active Power (P):")
print(mismatch_P)

print("\nMismatch in Reactive Power (Q):")
print(mismatch_Q)

Mismatch in Active Power (P):
0     0.257232
1    -0.027341
2     0.096987
3     0.572974
4     0.028627
5     0.065380
6     0.292280
7     0.052382
8    -0.108747
9    -0.136094
10    0.051110
11   -0.101865
Name: P_net, p.u., dtype: float64

Mismatch in Reactive Power (Q):
0     0.827205
1    -0.605070
2     0.291698
3     0.100060
4    -0.200866
5    -0.060401
6     0.025128
7    -0.030059
8     0.194339
9     0.126730
10   -0.056488
11    0.215130
Name: Q_net, p.u., dtype: float64


/var/folders/3h/60tkw2nd3cl5jw41dhky8jcc0000gn/T/ipykernel_74153/1117921442.py:14: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  P_calc[i] = V_bus[i] * (np.sum(V_bus[j] * ((np.real(Ybus[i, j])) * np.cos(theta[i]-theta[j]) \
/var/folders/3h/60tkw2nd3cl5jw41dhky8jcc0000gn/T/ipykernel_74153/1117921442.py:17: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  Q_calc[i] = V_bus[i] * (np.sum(V_bus[j] * ((np.real(Ybus[i, j])) * np.sin(theta[i]-theta[j]) \


In [246]:
# for P, eliminate Slack busses
# for Q, eleminate Slack and PV

non_S_indices = [i for i in range(len(mismatch_P)) if bus_data.iloc[i]['Type'] != 'S']

# Keeping only rows where the 'Type' is not 'S'
mismatch_P_filtered = mismatch_P[non_S_indices]

print(mismatch_P_filtered)

#remove bother 
D_indices = [i for i in range(len(mismatch_Q)) if bus_data.iloc[i]['Type'] == 'D']

# Keeping only rows where the 'Type' is 'G'
mismatch_Q_filtered = mismatch_Q[D_indices]

print(mismatch_Q_filtered)  # Updated mismatch_Q without rows where Type is 'G'


1    -0.027341
2     0.096987
3     0.572974
4     0.028627
5     0.065380
6     0.292280
7     0.052382
8    -0.108747
9    -0.136094
10    0.051110
11   -0.101865
Name: P_net, p.u., dtype: float64
1    -0.605070
3     0.100060
4    -0.200866
5    -0.060401
6     0.025128
7    -0.030059
10   -0.056488
Name: Q_net, p.u., dtype: float64


In [251]:
mismatch_array = np.concatenate((mismatch_P_filtered,mismatch_Q_filtered))
mismatch_array_indices = np.concatenate((non_S_indices, D_indices))
print(mismatch_array)
print(mismatch_array_indices)

[-0.02734074  0.09698669  0.57297383  0.02862718  0.06538008  0.29227984
  0.0523823  -0.10874737 -0.13609416  0.05110968 -0.10186511 -0.60507004
  0.10006025 -0.200866   -0.06040077  0.02512849 -0.03005887 -0.0564884 ]
[ 1  2  3  4  5  6  7  8  9 10 11  1  3  4  5  6  7 10]


In [253]:
# Count the number of PQ and PV buses
num_PQ_buses = len([1 for i in range(num_buses) if bus_data.iloc[i]['Type'] == 'D'])
num_PV_buses = len([1 for i in range(num_buses) if bus_data.iloc[i]['Type'] == 'G'])

# Initialize Jacobian matrix
J_size = 2 * num_PQ_buses + num_PV_buses
Jacobian = np.zeros((J_size, J_size))

# Assuming G and B are the real and imaginary parts of Ybus respectively
G = np.real(Ybus)
B = np.imag(Ybus)

# Calculate each element of the Jacobian matrix
for i in range(J_size):
    for j in range(J_size):
        if i == j:
            V_bus[i] * V_bus[j] (np.sum(
        # Calculate J_11, J_12, J_21, J_22 based on the formulas
        # and populate the Jacobian matrix accordingly
        # Make sure to account for i = j (diagonal elements) and i != j (off-diagonal elements)
        # Use the formulas provided above
        # ...

In [261]:
G = np.real(Ybus)
B = np.imag(Ybus)

num_PQ_buses = len([1 for i in range(num_buses) if bus_data.iloc[i]['Type'] == 'D'])
num_PV_buses = len([1 for i in range(num_buses) if bus_data.iloc[i]['Type'] == 'G'])
print(num_PQ_buses)
print(num_PV_buses)
print ('Jacobian')
    
# Function to populate the Jacobian matrix
def populate_jacobian(V_bus, theta, Ybus, G, B, non_S_indices, D_indices):
    n_pq = len(non_S_indices)
    n_pv = len(D_indices)
    J_size = 2 * n_pq + n_pv
    Jacobian = np.zeros((J_size, J_size), dtype=np.complex128)

    # Map bus numbers to Jacobian indices
    bus_to_idx = {bus: idx for idx, bus in enumerate(non_S_indices + D_indices)}

    # Populate J_11 and J_12 sub-matrices
    for i in range(n_pq):
        bus_i = non_S_indices[i]

        for j in range(n_pq):
            bus_j = non_S_indices[j]
            if i == j:  # Diagonal elements
                Jacobian[i, j] = -Q_calc[bus_i] - (V_bus[bus_i]**2 * B[bus_i, bus_i])
            else:  # Off-diagonal elements
                Jacobian[i, j] = -V_bus[bus_i] * V_bus[bus_j] * \
                                 (G[bus_i, bus_j] * np.sin(theta[bus_i] - theta[bus_j]) - \
                                  B[bus_i, bus_j] * np.cos(theta[bus_i] - theta[bus_j]))

            # Populate J_12
            if i == j:
                Jacobian[i, n_pq + j] = P_calc[bus_i] / V_bus[bus_i] + V_bus[bus_i] * G[bus_i, bus_i]
            else:
                Jacobian[i, n_pq + j] = V_bus[bus_i] * \
                                        (G[bus_i, bus_j] * np.cos(theta[bus_i] - theta[bus_j]) + \
                                         B[bus_i, bus_j] * np.sin(theta[bus_i] - theta[bus_j]))

    # Populate J_21 and J_22 sub-matrices
    for i in range(n_pq):
        bus_i = non_S_indices[i]

        for j in range(n_pq):
            bus_j = non_S_indices[j]
            # Populate J_21
            if i == j:
                Jacobian[n_pq + i, j] = P_calc[bus_i] - (V_bus[bus_i]**2 * G[bus_i, bus_i])
            else:
                Jacobian[n_pq + i, j] = V_bus[bus_i] * V_bus[bus_j] * \
                                        (G[bus_i, bus_j] * np.cos(theta[bus_i] - theta[bus_j]) + \
                                         B[bus_i, bus_j] * np.sin(theta[bus_i] - theta[bus_j]))

            # Populate J_22
            if i == j:
                Jacobian[n_pq + i, n_pq + j] = Q_calc[bus_i] / V_bus[bus_i] - V_bus[bus_i] * B[bus_i, bus_i]
            else:
                Jacobian[n_pq + i, n_pq + j] = V_bus[bus_i] * \
                                               (G[bus_i, bus_j] * np.sin(theta[bus_i] - theta[bus_j]) - \
                                                B[bus_i, bus_j] * np.cos(theta[bus_i] - theta[bus_j]))

    return Jacobian

# Assuming PQ_indices and PV_indices are the indices of PQ and PV buses respectively
Jacobian = populate_jacobian(V_bus, theta, Ybus, G, B, non_S_indices, D_indices)

# Print Jacobian matrix for inspection
print(Jacobian)

11
4
Jacobian
[[ 34.38129267+0.j   4.90140973+0.j   5.11583833+0.j   8.49043463+0.j
   -0.        +0.j  -0.        +0.j  -0.        +0.j  -0.        +0.j
   -0.        +0.j  -0.        +0.j  -0.        +0.j  14.13664899+0.j
   -1.13501919+0.j  -1.68603315+0.j  -6.54480579+0.j   0.        +0.j
    0.        +0.j   0.        +0.j   0.        +0.j   0.        +0.j
    0.        +0.j   0.        +0.j   0.        +0.j   0.        +0.j
    0.        +0.j   0.        +0.j   0.        +0.j   0.        +0.j
    0.        +0.j]
 [  4.90140973+0.j   5.26793942+0.j   0.36652969+0.j  -0.        +0.j
   -0.        +0.j  -0.        +0.j  -0.        +0.j  -0.        +0.j
   -0.        +0.j  -0.        +0.j  -0.        +0.j  -1.16339467+0.j
    2.66286919+0.j  -1.43607287+0.j   0.        +0.j   0.        +0.j
    0.        +0.j   0.        +0.j   0.        +0.j   0.        +0.j
    0.        +0.j   0.        +0.j   0.        +0.j   0.        +0.j
    0.        +0.j   0.        +0.j   0.        +0.j   0

In [248]:
'''
Z=1/y. 
if we delete one of the lines, 1/2 R. 
Cont. plan1 --
'''

'\nZ=1/y. \nif we delete one of the lines, 1/2 R. \nCont. plan1 --\n'

In [249]:
#when doing jacobian they have to be reduced. 